In [ ]:
%load core.py

In [2]:
# %load core.py
import os
import time
import gc
import time
import json

import numpy as np
from sklearn import decomposition
import umap.umap_ as umap

import config
import src.utils as utils
import src.bitops as bitops
import src.colour as colour
import src.clustering as clustering

"""
INITIALISE data TO WHOLE MAP EVEN IF USING EARLY STOP
.: huge memory req
can scale that down

also consider throwing away some, scaling down floats/ints etc

"""


"""
Parses spectrum-by-pixel maps from IXRF XFM

- parses binary .GeoPIXE files
- extracts pixel parameters
- projects spectra onto simple RGB channels
- displays as RGB

./data has example dataset

SPEED
                t/px
reading only:   0.00014 
colourmap:      0.0078
read and clust  0.001296 

"""

#-----------------------------------
#CLASSES
#-----------------------------------

#-----------------------------------
#INITIALISE
#-----------------------------------

starttime = time.time()             #init timer
chan=np.arange(0,config.NCHAN)      #channels
energy=chan*config.ESTEP            #energy list

#-----------------------------------
#MAIN START
#-----------------------------------

#check filetype is recognised - currently only accepts .GeoPIXE
if config.FTYPE == ".GeoPIXE":
    f = os.path.join(config.wdir,config.infile)
    fname = os.path.splitext(os.path.basename(f))[0]

    print("opening .geo:",fname)
else: 
    print(f'FATAL: filetype {config.FTYPE} not recognised')
    exit()

print(
    "---------------------------\n"
    "EXTRACTING SPECTRA\n"
    "---------------------------"
)

#open the datafile 
with open(f, mode='rb') as file: # rb = read binary
    
    #generate bytestream
    stream = file.read()         #NB. to read in chunks, add chunk size as read(SIZE)
    streamlen=len(stream)

    print(f"filesize: {streamlen} (bytes)")

    headerlen=bitops.binunpack(stream,0,"<H")[0]

    #check for header
    #   pixels start with "DP" (=20550 as <uint16)
    #   if we find this immediately, header is zero length
    #provided header is present
    #   read params from header
    if headerlen == 20550:
        print("WARNING: no header found")
        headerlen=0
        mapx=config.MAPX
        mapy=config.MAPY
        print("WARNING: map dimensions not found")
        print(f"-------using defaults {mapx},{mapy}")
    else:
        """
        if header present, read as json
        https://stackoverflow.com/questions/40059654/python-convert-a-bytes-array-into-json-format
        """
        #pull slice of byte stream corresponding to header
        #   bytes[0-2]= headerlen
        #   headerlen doesn't include trailing '\n' '}', so +2
        headerstream=stream[2:headerlen+2]
        #read it as utf8
        headerstream = headerstream.decode('utf8')
        
        #load into dictionary via json builtin
        headerdict = json.loads(headerstream)

        #create a human-readable dump for debugging
        headerdump = json.dumps(headerdict, indent=4, sort_keys=False)
        
        #get params
        mapx=headerdict['File Header']['Xres']  #map dimension x
        mapy=headerdict['File Header']['Yres']  #map dimension y

    #assign map size based on dimensions
    totalpx=mapx*mapy     

    #print run params
    print(f"header length: {headerlen} bytes")
    print(f"map dimensions x,y = {mapx},{mapy} px")

    #   if we are skipping some of the file
    #       assign the ratio and adjust totalpx
    if config.SHORTRUN:
        skipratio=config.shortpct/100
        trunc_y=int(np.ceil(mapy*skipratio))
        totalpx=mapx*trunc_y
        print(f"SHORT RUN: ending at {skipratio*100} %")

    print(f"pixels expected: {totalpx}")
    print("---------------------------")

    if config.FORCEREAD:
        #assign starting pixel index 
        idx=headerlen+2 #legnth of header + 2 bytes

        #initialise pixel param arrays
        pxlen=np.zeros(totalpx,dtype=np.uint16)
        xidx=np.zeros(totalpx,dtype=np.uint16)
        yidx=np.zeros(totalpx,dtype=np.uint16)
        det=np.zeros(totalpx,dtype=np.uint16)
        dt=np.zeros(totalpx,dtype=np.uint16)
        
        if config.DOCOLOURS == True:
            #initalise pixel colour arrays
            rvals=np.zeros(totalpx)
            gvals=np.zeros(totalpx)
            bvals=np.zeros(totalpx)
            totalcounts=np.zeros(totalpx)

        #initialise data array
        data=np.zeros((totalpx,config.NCHAN),dtype=np.uint16)

        i=0 #pixel counter
        j=0 #row counter

        #loop through pixels
        while idx < streamlen:

            #print pixel index every row px
            if i % mapx == 0: 
                print(f"Row {j}/{mapy} at pixel {i}, byte {idx} ({100*idx/streamlen:.1f} %)")
                j+=1

            #read pixel record into spectrum and header param arrays, 
            # + reassign index at end of read
            outchan, counts, pxlen[i], xidx[i], yidx[i], det[i], dt[i], idx = bitops.readpxrecord(idx, stream)

            #fill gaps in spectrum 
            #   (ie. add 0s for all missing chans)
            outchan, counts = utils.gapfill(outchan,counts, config.NCHAN)

            #warn if recieved channel list is different length to chan array
            if len(outchan) != len(chan):
                print("WARNING: unexpected length of channel list")
        
            #assign counts into data array - 
            data[i,:]=counts

            #build colours if required
            if config.DOCOLOURS == True: rvals[i], bvals[i], gvals[i], totalcounts[i] = colour.spectorgb(energy, counts)
            
            #if pixel index greater than expected no. pixels based on map dimensions
            #   end if we are doing a truncated run
            #   else throw a warning
            if i >= (totalpx-1):
                if (config.SHORTRUN == True):   #i > totalpx is expected for short run
                    print("ending at:", idx)
                    idx=streamlen+1
                    break 
                else:
                    print(f"WARNING: pixel count {i} exceeds expected map size {totalpx}")
            i+=1

        runtime = time.time() - starttime

        if config.SAVEPXSPEC:
            print(f"saving spectrum-by-pixel to file")
            np.savetxt(os.path.join(config.odir,  config.savename + ".dat"), data, fmt='%i')
        
        np.savetxt(os.path.join(config.odir, "pxlen.txt"), pxlen, fmt='%i')
        np.savetxt(os.path.join(config.odir, "xidx.txt"), xidx, fmt='%i')
        np.savetxt(os.path.join(config.odir, "yidx.txt"), yidx, fmt='%i')
        np.savetxt(os.path.join(config.odir, "detector.txt"), det, fmt='%i')
        np.savetxt(os.path.join(config.odir, "dt.txt"), dt, fmt='%i')


        print(
        "---------------------------\n"
        "MAP COMPLETE\n"
        "---------------------------\n"
        f"pixels expected (X*Y): {totalpx}\n"
        f"pixels found: {i}\n"
        f"total time: {round(runtime,2)} s\n"
        f"time per pixel: {round((runtime/i),6)} s\n"
        "---------------------------"
    )
    else:
        print("loading from file", config.savename)
        data = np.loadtxt(os.path.join(config.odir, config.savename), dtype=np.uint16)
        pxlen=np.loadtxt(os.path.join(config.odir, "pxlen.txt"), dtype=np.uint16)
        xidx=np.loadtxt(os.path.join(config.odir, "xidx.txt"), dtype=np.uint16)
        yidx=np.loadtxt(os.path.join(config.odir, "yidx.txt"), dtype=np.uint16)
        det=np.loadtxt(os.path.join(config.odir, "detector.txt"), dtype=np.uint16)
        dt=np.loadtxt(os.path.join(config.odir, "dt.txt"), dtype=np.uint16)
        print("loaded successfully", config.savename)
    "---------------------------\n"
    "Memory usage:\n"
    "---------------------------\n"
    utils.varsizes(locals().items())



    #clear the bytestream from memory
    del stream
    gc.collect()

    if config.DOCOLOURS == True:
        rgbarray=colour.clcomplete(rvals, gvals, bvals, totalcounts)
        colour.clshow(rgbarray)

    print("DOCLUST", config.DOCLUST)
    if config.DOCLUST:
        embedding, clusttimes = clustering.reduce(data)
        categories = clustering.dokmeans(embedding, totalpx)

        clustaverages=np.zeros([len(clustering.reducers),config.nclust,config.NCHAN])

        for i in range(len(clustering.reducers)):
            redname=clustering.getredname(i)
            clustaverages[i]=clustering.sumclusters(data, categories[i])
            
            for j in range(config.nclust):
                print(f'saving reducer {redname} cluster {j} with shape {clustaverages[i,j,:].shape}')
                np.savetxt(os.path.join(config.odir, "sum_" + redname + "_" + str(j) + ".txt"), np.c_[energy, clustaverages[i,j,:]], fmt=['%1.3e','%1.6e'])
            
            print(f'saving combined file for {redname}')
            np.savetxt(os.path.join(config.odir, "sum_" + redname + ".txt"), np.c_[energy, clustaverages[i,:,:].transpose(1,0)], fmt='%1.5e')             
            #plt.plot(energy, clustaverages[i,j,:])
        clustering.clustplt(embedding, categories, mapx, clusttimes)



print("CLEAN EXIT")


"""
snip background
https://stackoverflow.com/questions/57350711/baseline-correction-for-spectroscopic-data
"""

/home/lachlan/.pyenv/versions/venv_readoutxfm/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


script: /home/lachlan/CODEBASE/ReadoutXFM/config.py
script path: /home/lachlan/CODEBASE/ReadoutXFM
data path: /home/lachlan/CODEBASE/ReadoutXFM/data
---------------
opening .geo: geo_ln_chle
---------------------------
EXTRACTING SPECTRA
---------------------------
filesize: 198419529 (bytes)
header length: 1503 bytes
map dimensions x,y = 400,169 px
pixels expected: 67600
---------------------------
Row 0/169 at pixel 0, byte 1505 (0.0 %)
Row 1/169 at pixel 400, byte 1172821 (0.6 %)
Row 2/169 at pixel 800, byte 2348429 (1.2 %)
Row 3/169 at pixel 1200, byte 3522545 (1.8 %)
Row 4/169 at pixel 1600, byte 4701725 (2.4 %)
Row 5/169 at pixel 2000, byte 5886737 (3.0 %)
Row 6/169 at pixel 2400, byte 7080321 (3.6 %)
Row 7/169 at pixel 2800, byte 8286661 (4.2 %)
Row 8/169 at pixel 3200, byte 9493117 (4.8 %)
Row 9/169 at pixel 3600, byte 10700801 (5.4 %)
Row 10/169 at pixel 4000, byte 11911537 (6.0 %)
Row 11/169 at pixel 4400, byte 13117609 (6.6 %)
Row 12/169 at pixel 4800, byte 14319861 (7.2 %)


'\nsnip background\nhttps://stackoverflow.com/questions/57350711/baseline-correction-for-spectroscopic-data\n'

In [3]:
#import importlib
#importlib.reload(utils)

utils.varsizes(locals().items())

                          data: 528.1 MiB
                         pxlen: 132.1 KiB
                          xidx: 132.1 KiB
                          yidx: 132.1 KiB
                           det: 132.1 KiB
                            dt: 132.1 KiB
                          chan: 32.1 KiB
                        energy: 32.1 KiB
                       outchan: 32.1 KiB
                        counts: 32.1 KiB


In [4]:
import importlib
importlib.reload(clustering)




script: /home/lachlan/CODEBASE/ReadoutXFM/config.py
script path: /home/lachlan/CODEBASE/ReadoutXFM
data path: /home/lachlan/CODEBASE/ReadoutXFM/data
---------------


<module 'config' from '/home/lachlan/CODEBASE/ReadoutXFM/config.py'>

In [5]:
print("DOCLUST", config.DOCLUST)
if True:
    embedding, clusttimes = clustering.reduce(data)
    categories = clustering.dokmeans(embedding, totalpx)

    clustaverages=np.zeros([len(clustering.reducers),config.nclust,config.NCHAN])

    for i in range(len(clustering.reducers)):
        redname=clustering.getredname(i)
        clustaverages[i]=clustering.sumclusters(data, categories[i])
        
        for j in range(config.nclust):
            print(f'saving reducer {redname} cluster {j} with shape {clustaverages[i,j,:].shape}')
            np.savetxt(os.path.join(config.odir, "sum_" + redname + "_" + str(j) + ".txt"), np.c_[energy, clustaverages[i,j,:]], fmt=['%1.3e','%1.6e'])
        
        print(f'saving combined file for {redname}')
        np.savetxt(os.path.join(config.odir, "sum_" + redname + ".txt"), np.c_[energy, clustaverages[i,:,:].transpose(1,0)], fmt='%1.5e')             
        #plt.plot(energy, clustaverages[i,j,:])
    clustering.clustplt(embedding, categories, mapx, clusttimes)

DOCLUST False
REDUCER 1 of 2: PCA across 67600 elements
REDUCER 2 of 2: UMAP across 67600 elements
UMAP(min_dist=0.3, n_neighbors=30, verbose=True)
Thu Sep 29 22:36:45 2022 Construct fuzzy simplicial set
Thu Sep 29 22:36:46 2022 Finding Nearest Neighbors
Thu Sep 29 22:36:46 2022 Building RP forest with 18 trees
Thu Sep 29 22:36:57 2022 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	 4  /  16
	Stopping threshold met -- exiting after 4 iterations
Thu Sep 29 22:37:21 2022 Finished Nearest Neighbor Search
Thu Sep 29 22:37:24 2022 Construct embedding


Epochs completed:  27%| ██▋        54/200 [00:05]

In [ ]:
importlib.reload(clustering)

if config.DOCOLOURS == True:
    rgbarray=colour.clcomplete(rvals, gvals, bvals, totalcounts)
    colour.clshow(rgbarray)

print("DOCLUST", config.DOCLUST)
if config.DOCLUST:
    embedding, clusttimes = clustering.reduce(data)
    categories = clustering.dokmeans(embedding, totalpx)

    clustaverages=np.zeros([len(clustering.reducers),config.nclust,config.NCHAN])

    for i in range(len(clustering.reducers)):
        redname=clustering.getredname(i)
        clustaverages[i]=clustering.sumclusters(data, categories[i])
        
        for j in range(config.nclust):
            print(f'saving reducer {redname} cluster {j} with shape {clustaverages[i,j,:].shape}')
            np.savetxt(os.path.join(config.odir, "sum_" + redname + "_" + str(j) + ".txt"), np.c_[energy, clustaverages[i,j,:]], fmt=['%1.3e','%1.6e'])
        
        print(f'saving combined file for {redname}')
        np.savetxt(os.path.join(config.odir, "sum_" + redname + ".txt"), np.c_[energy, clustaverages[i,:,:].transpose(1,0)], fmt='%1.5e')             
        #plt.plot(energy, clustaverages[i,j,:])
    clustering.clustplt(embedding, categories, mapx, clusttimes)

In [ ]:
print(data.shape)

print(data[:,0:2500].shape)

rdata=data[:,0:2500]

#data=rdata

In [ ]:
import importlib
importlib.reload(clustering)

utils.varsizes(locals().items())


print("DOCLUST", config.DOCLUST)

if True:
    embedding, clusttimes = clustering.reduce(data)
    categories = clustering.dokmeans(embedding, totalpx)

    clustaverages=np.zeros([len(clustering.reducers),config.nclust,config.NCHAN])

    for i in range(len(clustering.reducers)):
        redname=clustering.getredname(i)
        clustaverages[i]=clustering.sumclusters(data, categories[i])
        
        for j in range(config.nclust):
            print(f'saving reducer {redname} cluster {j} with shape {clustaverages[i,j,:].shape}')
            np.savetxt(os.path.join(config.odir, "sum_" + redname + "_" + str(j) + ".txt"), np.c_[energy, clustaverages[i,j,:]], fmt=['%1.3e','%1.6e'])
        
        print(f'saving combined file for {redname}')
        np.savetxt(os.path.join(config.odir, "sum_" + redname + ".txt"), np.c_[energy, clustaverages[i,:,:].transpose(1,0)], fmt='%1.5e')             
        #plt.plot(energy, clustaverages[i,j,:])
    clustering.clustplt(embedding, categories, mapx, clusttimes)

In [ ]:
del embedding
del categories
del clustaverages
gc.collect()

In [ ]:
rdata = np.loadtxt(os.path.join(config.odir, "pxspec_dw.dat"), dtype=np.uint16)

print(data.shape)
print(rdata.shape)

In [ ]:
print(rdata[200,50:100])
print(data[200,50:100])

In [ ]:
utils.varsizes(locals().items())

In [ ]:
import matplotlib.pyplot as plt

workspec=clustaverages[1,4,:]

#plt.xscale("log")

plt.xlim(0.5,25)
plt.yscale("log")
plt.ylim(0.04,1000)
plt.plot(energy,workspec)




In [ ]:
"""
Baseline correction
"""
import pandas as pd
from scipy.signal import gaussian

#pybaselines project looks great
#https://pypi.org/project/pybaselines/

#alternately code from
#https://stackoverflow.com/questions/57350711/baseline-correction-for-spectroscopic-data

def baseline_correction4(raman_spectra,lam,p,niter=10):
    #according to "Asymmetric Least Squares Smoothing" by P. Eilers and H. Boelens
    number_of_spectra = raman_spectra.index.size

    #this is the code for the fitting procedure        
    L = len(raman_spectra.columns)
    w = np.ones(raman_spectra.shape[0]*raman_spectra.shape[1])

    D = sparse.block_diag(np.tile(sparse.diags([1,-2,1],[0,-1,-2],shape=(L,L-2)),number_of_spectra),format='csr')

    raman_spectra_flattened = raman_spectra.values.ravel()

    for jj in range(int(niter)):
        W = sparse.diags(w,format='csr')
        Z = W + lam * D.dot(D.transpose())
        z = spsolve(Z,w*raman_spectra_flattened,permc_spec='NATURAL')
        w = p * (raman_spectra_flattened > z) + (1-p) * (raman_spectra_flattened < z)
    #end of fitting procedure

    baseline_data = pd.DataFrame(z.reshape(number_of_spectra,-1),index=raman_spectra.index,columns=raman_spectra.columns)
    return baseline_data

In [ ]:
import struct 
import json

"""
https://stackoverflow.com/questions/40059654/python-convert-a-bytes-array-into-json-format
"""

headerstream=stream[2:headerlen+2]

headerstream = headerstream.decode('utf8')
"""
print(header[-100])
print("--------------------------")
print(jsonheader)
print("--------------------------")
#print(jsonheader[1300:1600])
#print("--------------------------")
print(jsonheader[1450:1480])
print(len(jsonheader))
"""

headerdict = json.loads(headerstream)
headerdump = json.dumps(headerdict, indent=4, sort_keys=False)

print(headerdump)
print("KEYS")
print(headerdict.keys())
nmapx=headerdict['File Header']['Xres']
nmapy=headerdict['File Header']['Yres']

print(nmapx,nmapy)



In [ ]:
import json

print(f)


with open(f) as file: # rb = read binary
  
    # returns JSON object as 
    # a dictionary
    jsondata = json.load(file)
    
    # Iterating through the json
    # list
    for i in jsondata['emp_details']:
        print(i)
    
    # Closing file
    #f.close()